Deliverable 1 Instructions
1. Generate a set of 2,000 random latitudes and longitudes
2. Retrieve the nearest city using the `citipy` module
3. Perform an API call with the OpenWeatherMap with below information:
    - Latitude and longitude
    - Maximum temperature
    - Percent humidity
    - Percent cloudiness
    - Wind speed
    - Weather description (for example, clouds, fog, light rain, clear sky)
4. Add the Data to a new DataFrame
5. Export DataFrame as a CSV file

In [1]:
# Import the dependencies.
import pandas as pd
import numpy as np

## 1. Generate a set of 2,000 random latitudes and longitudes

In [2]:
# Create a new set of 2,000 random latitudes and longitudes.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)
coordinates

[(52.51374841427548, 24.443158852849137),
 (-40.45178684811262, 18.11461322820557),
 (-22.661359338888687, 51.517165885633005),
 (73.64899794832516, -113.55864906772739),
 (-9.462278468727135, 34.326527526227125),
 (-42.989940707760724, 131.5233466731118),
 (-68.11413592398453, -167.20653736955833),
 (-76.61551732538989, 9.08450836019611),
 (36.76484969811152, -148.29410148570275),
 (-75.34284910427853, 85.77642794566594),
 (-64.07207086554325, 136.540929894893),
 (-35.49837093650477, -115.71816822481597),
 (19.651393566654207, 126.14963092141278),
 (0.5490268101078897, 86.99576572204154),
 (-38.029003481557076, 111.20167730865961),
 (89.67657747518672, -84.45525232947732),
 (19.07748589072483, -152.7706065972053),
 (15.554217352036872, -32.834839693453915),
 (-40.25957530877422, -80.88500817117954),
 (-5.2418101962282435, -160.51643245519335),
 (31.299875116483022, 17.217738131549794),
 (-62.989606167097136, -157.79247067840893),
 (-47.1588019260928, 49.969769226505065),
 (-40.3723197

## 2. Retrieve the nearest city using the `citipy` module

In [4]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [5]:
# Create a list for holding the cities.
cities = []
# Identify th enearest city for each atitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

755

## 3. Perform an API call with the OpenWeatherMap

In [12]:
# Import the datetime module from the datetime library.
from datetime import datetime
# Import the requests library.
import requests
# Import the API key.
from config import weather_api_key2

In [16]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key2
url

'http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=5b41765da1c99dd66d592ddc60d741d0'

In [17]:
city_url = url + "&q=" + "Boston"
city_weather = requests.get(city_url).json()
city_weather

{'coord': {'lon': -71.0598, 'lat': 42.3584},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 47.19,
  'feels_like': 42.55,
  'temp_min': 45,
  'temp_max': 48,
  'pressure': 1011,
  'humidity': 42},
 'visibility': 10000,
 'wind': {'speed': 10, 'deg': 344, 'gust': 18.01},
 'clouds': {'all': 55},
 'dt': 1617598113,
 'sys': {'type': 3,
  'id': 2013408,
  'country': 'US',
  'sunrise': 1617617956,
  'sunset': 1617664442},
 'timezone': -14400,
 'id': 4930956,
 'name': 'Boston',
 'cod': 200}

In [18]:
city_weather['weather'][0]['description']

'broken clouds'

In [19]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1


# Loop through all the cities in the list.
for i, city in enumerate(cities):
    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_description = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | kobryn
Processing Record 2 of Set 1 | hermanus
Processing Record 3 of Set 1 | mananjary
Processing Record 4 of Set 1 | yellowknife
Processing Record 5 of Set 1 | mwaya
Processing Record 6 of Set 1 | port lincoln
Processing Record 7 of Set 1 | avarua
Processing Record 8 of Set 1 | kahului
Processing Record 9 of Set 1 | busselton
Processing Record 10 of Set 1 | new norfolk
Processing Record 11 of Set 1 | rikitea
Processing Record 12 of Set 1 | san vicente
Processing Record 13 of Set 1 | hambantota
Processing Record 14 of Set 1 | qaanaaq
Processing Record 15 of Set 1 | hilo
Processing Record 16 of Set 1 | porto novo
Processing Record 17 of Set 1 | ancud
Processing Record 18 of Set 1 | samusu
City not found. Skipping...
Processing Record 19 of Set 1 | surt
Processing Record 20 of Set 1 | mataura
Processing Record 21 of Set 1 | tsihombe
City not found. Skipping...
Processing Record 22 of Set 1 | rio b

Processing Record 38 of Set 4 | komsomolskiy
Processing Record 39 of Set 4 | mys shmidta
City not found. Skipping...
Processing Record 40 of Set 4 | pangnirtung
Processing Record 41 of Set 4 | broome
Processing Record 42 of Set 4 | daru
Processing Record 43 of Set 4 | torbay
Processing Record 44 of Set 4 | ati
Processing Record 45 of Set 4 | griffith
Processing Record 46 of Set 4 | san pedro
Processing Record 47 of Set 4 | erenhot
Processing Record 48 of Set 4 | clyde river
Processing Record 49 of Set 4 | solnechnyy
Processing Record 50 of Set 4 | black river
Processing Record 1 of Set 5 | mtsamboro
Processing Record 2 of Set 5 | aykino
Processing Record 3 of Set 5 | arlit
Processing Record 4 of Set 5 | morristown
Processing Record 5 of Set 5 | aykhal
Processing Record 6 of Set 5 | seymchan
Processing Record 7 of Set 5 | thinadhoo
Processing Record 8 of Set 5 | hasaki
Processing Record 9 of Set 5 | metro
Processing Record 10 of Set 5 | nueva armenia
Processing Record 11 of Set 5 | poch

Processing Record 29 of Set 8 | kalmunai
Processing Record 30 of Set 8 | toora-khem
Processing Record 31 of Set 8 | narsaq
Processing Record 32 of Set 8 | kurumkan
Processing Record 33 of Set 8 | bosaso
Processing Record 34 of Set 8 | broken hill
Processing Record 35 of Set 8 | luganville
Processing Record 36 of Set 8 | boende
Processing Record 37 of Set 8 | nador
Processing Record 38 of Set 8 | springdale
Processing Record 39 of Set 8 | jadu
Processing Record 40 of Set 8 | jiddah
City not found. Skipping...
Processing Record 41 of Set 8 | paracho
City not found. Skipping...
Processing Record 42 of Set 8 | khasan
Processing Record 43 of Set 8 | jinchang
Processing Record 44 of Set 8 | chitral
Processing Record 45 of Set 8 | stjordalshalsen
Processing Record 46 of Set 8 | bitung
Processing Record 47 of Set 8 | tiksi
Processing Record 48 of Set 8 | saint-augustin
Processing Record 49 of Set 8 | byron bay
Processing Record 50 of Set 8 | vittoria
Processing Record 1 of Set 9 | sokoni
Proce

Processing Record 16 of Set 12 | yumen
Processing Record 17 of Set 12 | half moon bay
Processing Record 18 of Set 12 | meulaboh
Processing Record 19 of Set 12 | ashcroft
Processing Record 20 of Set 12 | fort morgan
Processing Record 21 of Set 12 | tual
Processing Record 22 of Set 12 | anori
Processing Record 23 of Set 12 | ramjibanpur
Processing Record 24 of Set 12 | mmabatho
Processing Record 25 of Set 12 | mitu
Processing Record 26 of Set 12 | badarpur
Processing Record 27 of Set 12 | cuencame
Processing Record 28 of Set 12 | nanchang
Processing Record 29 of Set 12 | toliary
City not found. Skipping...
Processing Record 30 of Set 12 | nioro
Processing Record 31 of Set 12 | jujuy
City not found. Skipping...
Processing Record 32 of Set 12 | felidhoo
City not found. Skipping...
Processing Record 33 of Set 12 | dekoa
City not found. Skipping...
Processing Record 34 of Set 12 | kununurra
Processing Record 35 of Set 12 | dharampuri
Processing Record 36 of Set 12 | natal
Processing Record 3

Processing Record 1 of Set 16 | wuwei
Processing Record 2 of Set 16 | huilong
Processing Record 3 of Set 16 | lavrentiya
Processing Record 4 of Set 16 | kolokani
Processing Record 5 of Set 16 | srednekolymsk
-----------------------------
Data Retrieval Complete      
-----------------------------


In [20]:
len(city_data)

691

## 4. Add the Data to a new DataFrame

In [21]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country,Date
0,Kobryn,52.2138,24.3564,32.00,86,0,8.95,clear sky,BY,2021-04-05 04:50:33
1,Hermanus,-34.4187,19.2345,63.00,91,99,4.00,overcast clouds,ZA,2021-04-05 04:50:33
2,Mananjary,-21.2167,48.3333,78.21,81,13,6.58,light rain,MG,2021-04-05 04:50:33
3,Yellowknife,62.4560,-114.3525,12.20,62,75,6.91,broken clouds,CA,2021-04-05 04:47:57
4,Mwaya,-9.5500,33.9500,73.72,83,84,0.31,light rain,TZ,2021-04-05 04:50:33
5,Port Lincoln,-34.7333,135.8667,69.75,60,0,10.13,clear sky,AU,2021-04-05 04:50:34
6,Avarua,-21.2078,-159.7750,80.60,89,75,5.75,light rain,CK,2021-04-05 04:48:23
7,Kahului,20.8947,-156.4700,71.60,83,40,17.27,scattered clouds,US,2021-04-05 04:50:34
8,Busselton,-33.6500,115.3333,82.00,55,2,1.99,clear sky,AU,2021-04-05 04:50:34
9,New Norfolk,-42.7826,147.0587,70.00,45,53,4.00,broken clouds,AU,2021-04-05 04:50:34


In [22]:
new_column_order = ['City', 'Country', 'Date', 'Lat', 'Lng', 'Max Temp',
                   'Humidity', 'Cloudiness', 'Wind Speed', 'Current Description']
city_data_df = city_data_df[new_column_order]
city_data_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Kobryn,BY,2021-04-05 04:50:33,52.2138,24.3564,32.00,86,0,8.95,clear sky
1,Hermanus,ZA,2021-04-05 04:50:33,-34.4187,19.2345,63.00,91,99,4.00,overcast clouds
2,Mananjary,MG,2021-04-05 04:50:33,-21.2167,48.3333,78.21,81,13,6.58,light rain
3,Yellowknife,CA,2021-04-05 04:47:57,62.4560,-114.3525,12.20,62,75,6.91,broken clouds
4,Mwaya,TZ,2021-04-05 04:50:33,-9.5500,33.9500,73.72,83,84,0.31,light rain
...,...,...,...,...,...,...,...,...,...,...
686,Wuwei,CN,2021-04-05 04:53:19,37.9281,102.6414,63.46,18,4,9.69,clear sky
687,Huilong,CN,2021-04-05 04:53:20,31.8111,121.6550,62.01,38,13,10.76,few clouds
688,Lavrentiya,RU,2021-04-05 04:53:20,65.5833,-171.0000,-3.10,86,14,3.36,few clouds
689,Kolokani,ML,2021-04-05 04:53:20,13.5728,-8.0339,77.86,9,50,6.26,scattered clouds


In [32]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")